In [1]:
import kagglehub
base_path = kagglehub.dataset_download("naveedhn/amazon-product-review-spam-and-non-spam")
print(base_path)  

C:\Users\Aybars\.cache\kagglehub\datasets\naveedhn\amazon-product-review-spam-and-non-spam\versions\1


In [ ]:
import pandas as pd
import glob
import os
import fastparquet

columns_to_keep = [
    'reviewerID', 
    'reviewText', 
    'summary', 
    'overall', 
    'helpful', 
    'unixReviewTime',
    'class'
]

json_files = glob.glob(os.path.join(base_path, "*/*.json"))

chunk_size = 100000
is_first_chunk = True

output_path = "C:/work environment/Projects/amazon-spam-review/data/raw/labeled_reviews.parquet"
# if os.path.exists(output_path):
#     print(f"Mevcut '{output_path}' dosyası siliniyor...")
#     os.remove(output_path)

for file in json_files:
  print(f"For döngüsü içinde {file}")
  try:
    category_name = os.path.basename(os.path.dirname(file))

    chunk_iterator = pd.read_json(file, lines=True, chunksize=chunk_size)

    for chunk in chunk_iterator:
      if "class" not in chunk.columns or chunk["class"].isnull().all():
        continue

      labeled_chunk = chunk.dropna(subset=["class"]).copy()

      if not labeled_chunk.empty:
          labeled_chunk["category"] = category_name
          final_columns = columns_to_keep + ['category']
          existing_columns = [col for col in final_columns if col in labeled_chunk.columns]

          final_chunk = labeled_chunk[existing_columns]

          if is_first_chunk:
            final_chunk.to_parquet(output_path, engine='fastparquet')
            is_first_chunk = False
          else:
            final_chunk.to_parquet(output_path, engine='fastparquet', append=True)
          

  except Exception as e:
    print(f"Error at {file} -error: {e}") 


print("\n🚀 Veri oluşturma işlemi tamamlandı!")

try:
    print("\nOluşturulan son dosyanın bilgileri okunuyor:")
    final_df = pd.read_parquet(output_path)
    final_df.info()
    print(f"\nToplam satır sayısı: {len(final_df)}")
    print("\nKategori dağılımı:")
    print(final_df['category'].value_counts())
except Exception as e:
    print(f"Dosya okunurken bir hata oluştu: {e}")

Mevcut 'C:/work environment/Projects/amazon-spam-review/data/raw/labeled_reviews.parquet' dosyası siliniyor...


PermissionError: [WinError 32] Dosya başka bir işlem tarafından kullanıldığından bu işlem dosyaya erişemiyor: 'C:/work environment/Projects/amazon-spam-review/data/raw/labeled_reviews.parquet'